In [1]:
import sys
import os

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller

import time
from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')

블로그 리뷰 url 크롤링

In [ ]:
# 여러 동에 대한 반복문
dong_list = ['연희동', '후암동', '망원동', '연남동', '성수동']
for dong in dong_list:
    # 각 동에 해당하는 카페 리스트 파일 읽기
    cafe_list_path = f'/Users/hana/MJU/3-2/빅데이터프로그래밍/BDP_Proj/cafelist/cafelist_{dong}.csv'
    cafe_list = pd.read_csv(cafe_list_path)
    cafe_list = cafe_list.drop("Unnamed: 0", axis=1)
    cafe_list = cafe_list.drop("cafe_type", axis=1)
    
    for cafe in cafe_list["name"]:
        dong_name = dong.replace('동', '')
        if dong_name + '점' in cafe:
            cafe = cafe.replace(dong_name + '점', '')
        
        cafe = dong_name + " " + cafe
        cafe_keywords = cafe.split()
    
        # 파일 경로 설정
        folder_path = f"BDP-Project/{dong}/"
        file_path = os.path.join(folder_path, f"{cafe}_블로그_리뷰.csv")
        if not os.path.exists(file_path):

            #크롬 웹브라우저 실행
            chrome_options = webdriver.ChromeOptions()
            
            chrome_options.add_argument("/Users/hana/.wdm/drivers/chromedriver/mac64/119.0.6045.105/chromedriver-mac-arm64/chromedriver")
            driver = webdriver.Chrome()
            driver.get(
                "https://search.naver.com"
            )
            time.sleep(1)
            
            cafe_keywords = cafe.split()
    
            #검색창에 "검색어" 검색
            element = driver.find_element(By.CSS_SELECTOR, 'div.search_input_box > input#query')
            element.send_keys(dong+" "+cafe)
            element.submit()
            time.sleep(1)
        
            #VIEW 클릭
            driver.find_element(By.LINK_TEXT, "VIEW").click()
            
            #옵션 클릭
            driver.find_element(By.LINK_TEXT, "옵션").click()
            
            #블로그 클릭
            driver.find_element(By.LINK_TEXT, "블로그").click()
        
            url_list = []
            title_list = []
        
            # 스크롤을 밑으로 내려주는 함수
            def scroll_down(driver):
                driver.execute_script("window.scrollTo(0, 99999999)")
                time.sleep(1)
            
                # n : 스크롤할 횟수 설정
                n = 5 #스크롤 1번당 글 30개씩 화면에 출력
                i = 0
                while i < n:
                    scroll_down(driver) #스크롤다운
                    i = i+1
            
            
            #URL_raw 크롤링
            articles = "a.title_link"
            article_raw = driver.find_elements(By.CSS_SELECTOR, articles)
            for article in article_raw:
                title = article.text
        
                if all(keyword in title for keyword in cafe_keywords):
                    url = article.get_attribute('href')
                    url_list.append(url)
                    title_list.append(title)
        
                else:
                    continue
                    
            # 10개 미만으로 게시글을 가질 경우 검색 결과 상위 10개 추출
            if (len(url_list) < 10):
                url_list = []
                title_list = []

                article_raw = article_raw[:10]
                
                #URL_raw 크롤링
                for article in article_raw:
                    url = article.get_attribute('href')
                    url_list.append(url)
                
                    title = article.text
                    title_list.append(title)

            print("")
            print(cafe)
            print('url 갯수: ',len(url_list))
            print('title 갯수: ',len(title_list))
            
            df = {}
            df = pd.DataFrame({'url':url_list, 'title':title_list})
            
            # 폴더가 존재하지 않으면 생성
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)

            # 파일명 지정 (i 변수를 사용하여 동적으로 파일명 생성)
            file_name = f"{cafe}_블로그_리뷰.csv"
            
            # 파일 전체 경로 생성
            file_path = os.path.join(folder_path, file_name)
            
            # 데이터프레임을 CSV 파일로 저장
            df.to_csv(file_path, index=False)
            driver.close()
            
        else:
            print(f"File {file_path} already exists. Skipping…\n")
    
print(f"Crawling for {dong} completed.\n")

블로그 리뷰 콘텐츠 크롤링

In [7]:

# ChromeOptions 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("/Users/hana/.wdm/drivers/chromedriver/mac64/119.0.6045.105/chromedriver-mac-arm64/chromedriver")

# 크롬 웹브라우저 실행
driver = webdriver.Chrome()
driver.get("https://search.naver.com")
time.sleep(1)

dong_list = ['망원동']

for dong in dong_list:
    # 폴더 경로 설정
    folder_path = f"/Users/najee/BigdataProgramming2023/cabojago/data/blog_review/blog_review_url_list/{dong}/"
    
    # 폴더 내 파일 목록 가져오기
    file_list = os.listdir(folder_path)
    
    # 파일 순차적으로 읽기
    for file_name in file_list:
        cafe_name = file_name.split('_')[0]
        
        
        # 결과 저장 파일 경로 읽기 
        result_folder_path = f"/Users/najee/BigdataProgramming2023/cabojago/data/blog_review/blog_review_result_list/{dong}/"
        in_result_file_path = os.path.join(result_folder_path, f"{cafe_name}_blog_content.csv")
        
        if not os.path.exists(in_result_file_path):
            print(file_name)
        
            # 파일 읽기 예시 (pandas를 사용하여 CSV 파일을 읽는 경우)
            if file_name.endswith(".csv"):
                file_path = os.path.join(folder_path, file_name)
                df = pd.read_csv(file_path)
                
                result_dict = {}
    
                number = len(df)
                
                for i in tqdm_notebook(range(number)):
                    # 글 띄우기
                    url = df['url'][i]
                    driver.get(url)
                    
                    try:
                        driver.switch_to.frame('mainFrame')
                
                        target_info = {}
                
                        # 제목 크롤링
                        overlays = ".se-module.se-module-text.se-title-text"
                        tit = driver.find_element(By.CSS_SELECTOR, overlays)
                        title = tit.text
                
                        # 내용 크롤링
                        overlays = ".se-main-container"
                        content = driver.find_element(By.CSS_SELECTOR, overlays)
                        cont = content.text
                        cont = cont.replace('\n', ' ')
                
                        # 딕셔너리에 담음
                        target_info['title'] = title
                        target_info['content'] = cont
                
                        # 전체 딕셔너리에 담음
                        result_dict[i] = target_info
                        time.sleep(1)
                
                        print(i, title)

                    except:
                        continue
                    
                print('수집글 갯수', len(result_dict))
                result = pd.DataFrame.from_dict(result_dict, orient='index')

                # 결과 저장 폴더가 존재하지 않으면 생성
                if not os.path.exists(result_folder_path):
                    os.makedirs(result_folder_path)

                # 파일명 지정 (i 변수를 사용하여 동적으로 파일명 생성)
                result_file_name = f"{cafe_name}_blog_content.csv"
                
                # 파일 전체 경로 생성
                result_file_path = os.path.join(result_folder_path, result_file_name)
                
                # 데이터프레임을 CSV 파일로 저장
                result.to_csv(result_file_path, index=False)
                print(" ")
                time.sleep(3)

        else:
            print(f"File {in_result_file_path} already exists. Skipping…\n")

# 마무리 작업
driver.close()
print("All files processed.")


File /Users/najee/BigdataProgramming2023/cabojago/data/blog_review/blog_review_result_list/망원동/망원동 604seoul_blog_content.csv already exists. Skipping…

File /Users/najee/BigdataProgramming2023/cabojago/data/blog_review/blog_review_result_list/망원동/망원동 79파운야드 망원본점_blog_content.csv already exists. Skipping…

File /Users/najee/BigdataProgramming2023/cabojago/data/blog_review/blog_review_result_list/망원동/망원동 9ram_blog_content.csv already exists. Skipping…

File /Users/najee/BigdataProgramming2023/cabojago/data/blog_review/blog_review_result_list/망원동/망원동 AIT_blog_content.csv already exists. Skipping…

File /Users/najee/BigdataProgramming2023/cabojago/data/blog_review/blog_review_result_list/망원동/망원동 Alive214_blog_content.csv already exists. Skipping…

File /Users/najee/BigdataProgramming2023/cabojago/data/blog_review/blog_review_result_list/망원동/망원동 BGT호두단팥빵 망원동점_blog_content.csv already exists. Skipping…

File /Users/najee/BigdataProgramming2023/cabojago/data/blog_review/blog_review_result_lis

  0%|          | 0/558 [00:00<?, ?it/s]

0 서울 프렌치토스트 맛집 : 망원동 키오스크 카페 오픈런
1 9월 일상 / 부천 추억여행(캔모아,즉떡) / 무지개 간식게임 / 망원동 나들이(키친갈매기,키오스크,진부책방스튜디오) / 뚝섬유원지 노을 / 추석일상
2 [키오스크] 나만 알고 싶은 망원동 프토 맛집 망원카페
3 늦게까지 하는 망원동 카페, 프렌치토스트 커피 맛집 키오스크
4 망원동 키오스크 카페 . 프렌치토스트 맛집 (블루베리 허니 크림치즈)
5 [카페] 프렌치토스트 맛집 망원동 키오스크(KIOSQUE)
6 [망원동] 키오스크 프렌치토스트 / 헤키 돈까스
7 [마포/망원동] 망원동 카페 추천! 프렌치토스트의 찐인 ‘키오스크’ (커피도 맛있음!)
8 망원동 키오스크토스트
9 [망원동]키오스크,프렌치토스트,디저트맛집,카페추천,전참시 카페,내돈내산
10 수상할 정도로 맛있는 프렌치토스트 망원동 키오스크
11 [망원동] 전참시토스트맛집 "키오스크"
12 서울 마포구 망원동 : 사랑도 디저트도 결국 타이밍 '키오스크'
13 망원동 키오스크 프렌치토스트 : 내돈내산 서울 디저트 존맛 전참시 핫플 맛집 카페
14 합정 망원동 맛집 카페 키오스크 (망원 프렌치토스트 맛집)
15 망원동 키오스크 프렌치토스트와 커피 카페
16 [ 키오스크 ] 망원동 - 인생 프렌치토스트 🍞🤎 , 망원카페
17 망원동 카페 어쩌다가게 '키오스크' 프렌치 토스트 맛집
18 230401 _ 키오스크 프렌치토스트 @어쩌다가게, 망원동
19 망원동 키오스크, 맛있는 위로의 프렌치토스트
20 [맛집 탐방] 망원동 프렌치 토스트 맛집 ‘키오스크’
21 오랜만에 망원동 키오스크 라임 프렌치토스트
22 9가지 맛 프토가 요기또..☆ 망원동 카페 키오스크(KIOSQUE) (feat.블루베리 프렌치토스트)
23 망원동 겉바속촉 프렌치토스트 맛집 디저트 카페 *키오스크*
24 [서울카페/망원동카페] 작은 카페, 프렌치토스트 맛집 ‘키오스크’ (kiosque)
25 망원동 키오스크 _ 프렌치 토스트 맛집, 망원동 디저트맛집, 주차 내돈내산
26 망원동 

  0%|          | 0/10 [00:00<?, ?it/s]

0 망원동 디저트 카페 ‘키토빵앗간’ 저탄수 저당 글루텐프리 빵집, 다이어트 당뇨 임당간식(택배)
1 망원동 빵집 키토빵앗간 : 상자째 들고 온 글루텐프리 빵들 후기(top3)
2 망원동 카페 키토빵앗간 속세의 맛 키토 디저트 베이커리 빵집
3 시골쥐 서울 나들이 - 망원동 저탄수 글루텐프리 디저트 카페 키토빵앗간
4 망원동 골목을 걷다가 보면, 멈출 수밖에 없는_[키토빵앗간]
5 망원동 비건빵투어 후기_두두리두팡, 키토빵앗간
6 [먹어볼까 🥐] 망원동 글루텐프리 감성빵집 <키토빵앗간>
7 서울•망원동 | 무설탕 저탄수 글루텐프리 다이어트 디저트 추천 <키토빵앗간>
8 내돈내산 키토빵앗간 저탄수빵 글루텐프리빵 다이어트빵 무설탕빵 키토빵 다이어트빵 수제디저트 다이어터빵 망원동 빵집
9 [망원동 베이커리 맛집 추천!] 키토빵앗간 🍞
수집글 갯수 10
 
망원동 투고벅스_블로그_리뷰.csv


...

  0%|          | 0/1 [00:00<?, ?it/s]

0 프랜차이즈와 노하우 그리고 마케팅
수집글 갯수 1
 
망원동 퍼스널커피_블로그_리뷰.csv


  0%|          | 0/171 [00:00<?, ?it/s]

0 [망원동카페] 퍼스널슈페너와 버터바가 맛있는 퍼스널커피
1 망원동카페 <퍼스널커피> 인생라떼 찾았음!
2 [망원]내가 좋아하는 망원동 분위기_퍼스널커피/망리단길상권
3 버터바가 맛있었던 조용한 망원동카페 ‘퍼스널커피’(영업시간/유자 얼그레이/ 라떼맛집)
4 망리단길 망원동 카페 퍼스널커피 제대로 된 디저트 강자
5 망원동카페 :: 스콘이 맛있는 '퍼스널커피'
6 망원동카페 퍼스널커피 PERSONAL COFFEE
7 [망원역 카페] 퍼스널커피⎮망원동약과쿠키 맛집, 망원동버터바 맛집 (위치, 메뉴판)
8 [망원동 카페] 망리단길 스콘 맛집 조용한 카페 추천 :: 퍼스널커피
9 망원동 카페, 퍼스널커피 인생 버터바 (반려동물 동반)
10 크림치즈 약과쿠키 망원동 퍼스널커피
11 [퍼스널커피] 망원동 카페 // 디카페인 맛집
12 망원동 카페 쿠키맛집 퍼스널커피 n번째방문 찐후기
13 [망원동카페: 망원역] 망원동버터바 망원동약과쿠키, 퍼스널커피
14 [망원동 카페] 퍼스널커피/ 퍼스널 슈페너 맛집이네... 꾸덕한 크림이 존맛탱구리
15 망원동 혼자가기 좋은 카페 퍼스널커피(수제 약과쿠키, 버터바 맛집)
16 망리단길 퍼스널커피 망원동카페 민트초코 버터바
17 망원동 꾸덕한 버터바 와 고소한 크림 라떼 맛집 [ 퍼스널커피 ] 마포구 로스터리 카페
18 망원동 약과쿠키와 버터바 맛집 퍼스널커피
19 망원동카페 퍼스널커피 (Personal Coffee)
20 조용하고 깔끔해서 혼커하기 좋은 망원동 애견동반카페 퍼스널커피
21 퍼스널커피 예쁘고 맛있는 여유로운 망원동카페
22 서울 망원동 퍼스널커피 스콘까지 맛있는 망원동 카페
23 [마포,망원] 망원동카페 민초단이라면 지나칠 수 없는 곳. 민트초코 버터바가 있는 ‘퍼스널커피’
24 망원동 약과 쿠키 맛집! 꾸덕한 버터바가 있는 로스터리 카페, 퍼스널커피.
25 망원동카페) 퍼스널커피 PERSONAL COFFEE (feat. 퍼스널슈페너, 솔티 카라멜 버터바 맛집 / 로스터리 카페)
26 [망원] 커피&디저트 맛있는 망

KeyboardInterrupt: 